# LNN Scratchpad

Here I'll be messing with the [LNN Library](https://github.com/IBM/LNN) from IBM.  I want to recreate some [SWI](https://www.swi-prolog.org/) Prolog style queries as well work with their paradigm.  Hopefully this will be slightly useful to others playing with the library and, of course, to myself!

In [1]:
import lnn

## Socrates 

He's a man baby, and that makes him mortal.  In prolog this is pretty straightforward. In Prolog, the predicates are "backward" to me.

Nota bene: I understand the modern idiom is to use "human", but later on I want to mess with more predicates so I'm sticking with `man` for the moment.  Yay to trans rights, NB-rights, LGTQIA rights AND to concise coding.  Ain't about that right now.

You will need a [db](db.pl) file to load with rules in the standard `swipl` fashion and I won't fight that shit.


```
> [db].
True

mortal(socrates).
True
```

The LNN library is a bit more verbose. Also, they don't seem to have settled on a coding style guide yet, so I'm using the one from the [API documentation](https://ibm.github.io/LNN/usage.html) and not the [Tutorials](https://github.com/IBM/LNN/tree/master/tutorials).  Don't worry if makes 0 sense to you right now.

In [2]:
# Create a model
model = lnn.Model()

# Create variables. Here I follow the Prolog convention of capitals, they like lowercase
X = lnn.Variable('X')

# Create some Predicates (has variables).
Man, Mortal = lnn.Predicates('Man', 'Mortal')

# Axioms
Men_Are_Mortal = lnn.ForAll(X, lnn.Implies(Man(X), Mortal(X)))

# What we want to ask
who_dies = lnn.Exists(X, Mortal(X))

# Add knowledge to the model
model.add_knowledge(Man, Mortal, Men_Are_Mortal, who_dies)

# Now add data about particular objects / records / people
# Note this is classical logic, so the truth is basically just "TRUE" or "FALSE"
model.add_data({
    Man: {'Socrates': lnn.Fact.TRUE}
})

# RUN THAT BABY!!
model.infer()

# SHOW ME THE MONEY!!
model.print()


***************************************************************************
                                LNN Model

OPEN Exists: (∃0, Mortal(0))                                TRUE (1.0, 1.0)

AXIOM ForAll: (∀0, (Man(0) → Mortal(0)))                    TRUE (1.0, 1.0)

OPEN Implies: (Man(0) → Mortal(0)) 
'Socrates'                                                  TRUE (1.0, 1.0)

OPEN Predicate: Mortal 
'Socrates'                                                  TRUE (1.0, 1.0)

OPEN Predicate: Man 
'Socrates'                                                  TRUE (1.0, 1.0)

***************************************************************************


So the model agrees that Socrates is mortal and we've re-created the Prolog statements.  Now let's check out why this is so much more verbose than Prolog.

## Socrates Is Fluid

Now, I'm not here to teach you anything about [Real Valued Logic](https://en.wikipedia.org/wiki/%C5%81ukasiewicz_logic) other than it's a pretty integral part of [Neuro Symbolic Artificial Intelligence](https://en.wikipedia.org/wiki/Neuro-symbolic_AI), which happens to be my jam right now.  The idea is we can assign confidence to various kinda truthy statements, which means  *FUCK YEAH PROBABILITY THEORY*.  Get yourself a cup of coffee and a copy of [Durrett](https://bookshop.org/p/books/probability-theory-and-examples-rick-durrett/12965062?ean=9781108473682), super fun shit.

Now we let Socrates have his/her/their own identity.  Run the whole thing again to keep the variables clean.

In [3]:
# Create a model
model = lnn.Model()

# Create variables. Here I follow the Prolog convention of capitals, they like lowercase
X = lnn.Variable('X')

# Create some Predicates (has variables).
Man, Mortal = lnn.Predicates('Man', 'Mortal')

# Axioms
Men_Are_Mortal = lnn.ForAll(X, lnn.Implies(Man(X), Mortal(X)))

# What we want to ask
who_dies = lnn.Exists(X, Mortal(X))

# Add knowledge to the model
model.add_knowledge(Man, Mortal, Men_Are_Mortal, who_dies)

# Now add data about particular objects / records / people
model.add_data({
    Man: {'Socrates': (0.4, 0.7)}
})

# RUN THAT BABY!!
model.infer()

# SHOW ME THE MONEY!!
model.print()


***************************************************************************
                                LNN Model

OPEN Exists: (∃0, Mortal(0))                       APPROX_UNKNOWN (0.4, 1.0)

AXIOM ForAll: (∀0, (Man(0) → Mortal(0)))                    TRUE (1.0, 1.0)

OPEN Implies: (Man(0) → Mortal(0)) 
'Socrates'                                                  TRUE (1.0, 1.0)

OPEN Predicate: Mortal 
'Socrates'                                         APPROX_UNKNOWN (0.4, 1.0)

OPEN Predicate: Man 
'Socrates'                                         APPROX_UNKNOWN (0.4, 0.7)

***************************************************************************


So now Socrates is mortal if he is a Man, which he is questioning.  We don't have an answer if Socrates is not a Man, because we don't know if Women are mortal yet.

In [4]:
# Create a model
model = lnn.Model()

# Create variables. Here I follow the Prolog convention of capitals, they like lowercase
X = lnn.Variable('X')

# Create some Predicates (has variables).
Woman, Man, Mortal = lnn.Predicates('Woman', 'Man', 'Mortal')

# Axioms
Men_Are_Mortal = lnn.ForAll(X, lnn.Implies(Man(X), Mortal(X)))
Women_Are_Mortal = lnn.ForAll(X, lnn.Implies(Woman(X), Mortal(X)))

# What we want to ask
who_dies = lnn.Exists(X, Mortal(X))

# Add knowledge to the model
model.add_knowledge(Woman, Man, Mortal, Men_Are_Mortal, Women_Are_Mortal, who_dies)

# Now add data about particular objects / records / people
model.add_data({
    Man: {'Socrates': (0.4, 0.7), 'Plato': (0.8, 1.0)},
    Woman: {'Socrates': (0, 0.5)}
})

# RUN THAT BABY!!
model.infer()

# SHOW ME THE MONEY!!
model.print()


***************************************************************************
                                LNN Model

OPEN Exists: (∃0, Mortal(0))                                TRUE (1.0, 1.0)

AXIOM ForAll: (∀0, (Woman(0) → Mortal(0)))                  TRUE (1.0, 1.0)

OPEN Implies: (Woman(0) → Mortal(0)) 
'Socrates'                                                  TRUE (1.0, 1.0)
'Plato'                                                     TRUE (1.0, 1.0)

AXIOM ForAll: (∀0, (Man(0) → Mortal(0)))                    TRUE (1.0, 1.0)

OPEN Implies: (Man(0) → Mortal(0)) 
'Socrates'                                                  TRUE (1.0, 1.0)
'Plato'                                                     TRUE (1.0, 1.0)

OPEN Predicate: Mortal 
'Socrates'                                         APPROX_UNKNOWN (0.4, 1.0)
'Plato'                                              APPROX_TRUE (0.8, 1.0)

OPEN Predicate: Man 
'Socrates'                                         APPROX_UNKNOWN (0.4

If you ask me, this is pretty fun.  Socrates stil has a lot of questioning to do before he comes out as a `Mortal`.

# Short Skirt, Long Jacket

Turns out some men are just too picky for their own damn good, and the guys from Cake made a nice [satire](https://www.youtube.com/watch?v=X5KmB8Laemg)

In [5]:
# Create a model
model = lnn.Model()

# Create variables. Here I follow the Prolog convention of capitals, they like lowercase
X = lnn.Variable('X')


FingernailsShineLikeJustice, Skirt = lnn.Predicates("FingernailsShineLikeJustice", "LongSkirt")

# Axioms

# Predicates direct to model?  Seems easier
model.add_knowledge(
    FingernailsShineLikeJustice, LongSkirt,
    lnn.And(FingernailsShineLikeJustice, LongSkirt)
)

# Add some candidates (data)
model.add_data({
    FingernailsShineLikeJustice: {'Alice': lnn.Fact.TRUE}
})

# Run that Baby!
model.infer()

# SHOW ME THE MONEY!
model.print()


***************************************************************************
                                LNN Model

OPEN And: (FingernailsShineLikeJustice(0) ∧ LongSkirt(1)) 

OPEN Predicate: LongSkirt 

OPEN Predicate: FingernailsShineLikeJustice 
'Alice'                                                     TRUE (1.0, 1.0)

***************************************************************************


My life is going to be so much easier if we can get the models built by marshalling JSON.  I'm sure there is a way, but grrr..

In [4]:
# Create a model
model = lnn.Model()

# Create variables. Here I follow the Prolog convention of capitals, they like lowercase
X = lnn.Variable('X')

preds = [
    'FingernailsShineLikeJustice', 'ShortSkirt'
]


# Axioms

# Predicates direct to model?  Seems easier
model.add_knowledge(
    *[lnn.Predicate(p) for p in preds],
    lnn.And('FingernailsShineLikeJustice', 'ShortSkirt')
)

# Add some candidates (data)
model.add_data({
    FingernailsShineLikeJustice: {'Alice': lnn.Fact.TRUE}
})

# Run that Baby!
model.infer()

# SHOW ME THE MONEY!
model.print()

AttributeError: 'str' object has no attribute 'propositional'